# Objectif : Nettoyage, transformation, imputation, encodage et export pour modélisation

In [42]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# 1. Chargement des données EDA

In [43]:
base = pd.read_csv("eda.csv")

# 2. Suppression des colonnes inutiles pour la modélisation

In [44]:
colonnes_a_supprimer = [
    'reference_web', 'commentaire_indicateur', 'resultats_obtenus_lancement',
    'criteres_admissibilite', 'partenaires', 'processus_suivi', 'contexte',
    'axe', 'programme', 'resultats_attendus', 'resultats_obtenus_annee',
    'action_indicateurs', 'action_actions', 'action_merged',
    'source_financement', 'q__ges', 'mesure', 'nom_beneficiaire', 'id_beneficiaire'
]
base = base.drop(columns=[col for col in colonnes_a_supprimer if col in base.columns])

# 3. Traitement des valeurs manquantes

In [45]:
base['depense_reelle'] = base['depense_reelle'].fillna(0)
base['depense_prevue'] = base['depense_prevue'].fillna(base['depense_prevue'].median())
base['cible_indicateur'] = base['cible_indicateur'].fillna(0)
base['pourcentage_resultat'] = base['pourcentage_resultat'].fillna(0)
base['pourcentage_electrification'] = base['pourcentage_electrification'].fillna(0)

# Variables catégorielles : remplissage par "inconnu"

In [ ]:
vars_cat = ['type_indicateur', 'ministere', 'finalite', 'statut', 'categorie_beneficiaire_action']
for col in vars_cat:
    if col in base.columns:
        base[col] = base[col].fillna("inconnu")

# 5. Transformation : pourcentage_electrification en binaire

In [47]:
base['electrique'] = base['pourcentage_electrification'].apply(lambda x: 1 if x > 0 else 0)

# 6. Création de variables dérivées

In [48]:
base["ratio_resultat_cible"] = base["pourcentage_resultat"] / (base["cible_indicateur"] + 1e-6)
seuil = base["depense_prevue"].median()
base["gros_budget"] = (base["depense_prevue"] > seuil).astype(int)

# 7. Encodage des variables catégorielles avec LabelEncoder

In [49]:
enc = LabelEncoder()
for col in vars_cat:
    if col in base.columns:
        base[col + "_libelle"] = base[col]  # sauvegarde avant encodage
        base[col] = enc.fit_transform(base[col])

# 8. Normalisation des variables continues

In [50]:
scaler = StandardScaler()
base[['depense_prevue', 'cible_indicateur', 'pourcentage_resultat', 'ratio_resultat_cible']] = scaler.fit_transform(
    base[['depense_prevue', 'cible_indicateur', 'pourcentage_resultat', 'ratio_resultat_cible']]
)


# 9. Création du fichier prêt pour modélisation

In [51]:
colonnes_finales = [
    'depense_prevue', 'cible_indicateur', 'pourcentage_resultat', 'ratio_resultat_cible',
    'gros_budget', 'type_indicateur', 'ministere', 'ministere_libelle',
    'finalite', 'finalite_libelle', 'statut', 'statut_libelle',
    'categorie_beneficiaire_action', 'categorie_beneficiaire_action_libelle',
    'electrique', 'depassement'
]
df_final = base[colonnes_finales]
df_final.to_csv("pretraitement.csv", index=False)

print("\n✅ Données prétraitées affinées enregistrées sous pretraitement.csv avec libellés et enrichissements")
df_final.head()



✅ Données prétraitées affinées enregistrées sous pretraitement.csv avec libellés et enrichissements


,depense_prevue,cible_indicateur,pourcentage_resultat,ratio_resultat_cible,gros_budget,type_indicateur,ministere,ministere_libelle,finalite,finalite_libelle,statut,statut_libelle,categorie_beneficiaire_action,categorie_beneficiaire_action_libelle,electrique,depassement
0,5.738749,-0.091057,0.496012,-0.359706,1,1,3,MELCCFP,1,Atténuation,0,en cours,2,Citoyens,1,1
1,4.270889,-0.091057,0.496012,-0.359706,1,1,3,MELCCFP,1,Atténuation,0,en cours,2,Citoyens,1,0
2,3.149278,-0.091057,0.496012,-0.359706,1,1,3,MELCCFP,1,Atténuation,0,en cours,2,Citoyens,1,0
3,2.092774,-0.091057,0.496012,-0.359706,1,1,3,MELCCFP,1,Atténuation,0,en cours,2,Citoyens,1,0
4,2.489333,-0.091057,0.496012,-0.359706,1,1,3,MELCCFP,1,Atténuation,0,en cours,2,Citoyens,1,0
